## YOLOX dataset

In this notebook you can find code for splitting and transforming COTS dataset to YOLOX-type dataset. 

In [ ]:
"""CLI for dataframe splitting on train and test"""


from enum import Enum
from pathlib import Path
from typing import List, Tuple

import pandas as pd


class SplitType(Enum):
    video: str = 'video'
    length: str = 'length'


def _split_dataframe_by_videos(
    dataframe: pd.DataFrame, train_video_ids: List[int], val_video_ids: List[int]
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_dataframe = dataframe.loc[dataframe['video_id'].isin(train_video_ids)]
    val_dataframe = dataframe.loc[dataframe['video_id'].isin(val_video_ids)]

    return train_dataframe, val_dataframe


def _split_dataframe_by_length(
    dataframe: pd.DataFrame, val_length: float = 0.2
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    train_dataframes = []
    val_dataframes = []

    for unique_video_id in dataframe['video_id'].unique():
        dataframe_part: pd.DataFrame = dataframe.loc[
            dataframe['video_id'] == unique_video_id
        ]

        dataframe_part.sort_values(by='video_frame')

        train_part = dataframe_part.iloc[: -int(val_length * dataframe_part.shape[0])]
        val_part = dataframe_part.iloc[-int(val_length * dataframe_part.shape[0]) :]

        train_dataframes.append(train_part)
        val_dataframes.append(val_part)

    train_df = pd.concat(train_dataframes)
    val_df = pd.concat(val_dataframes)

    return train_df, val_df


def split_dataframe(
    original_dataframe_path: Path,
    train_name: str = 'train_part.csv',
    val_name: str = 'val_part.csv',
    split_type: SplitType = SplitType.video,
) -> None:
    dataframe = pd.read_csv(original_dataframe_path)

    if split_type == SplitType.video:
        train_dataframe, val_dataframe = _split_dataframe_by_videos(
            dataframe=dataframe, train_video_ids=[0, 1], val_video_ids=[2]
        )
    elif split_type == SplitType.length:
        train_dataframe, val_dataframe = _split_dataframe_by_length(
            dataframe=dataframe, val_length=0.3
        )
    else:
        raise ValueError('No such dataset type')

    train_dataframe.to_csv(train_name, index=False)
    val_dataframe.to_csv(val_name, index=False)


In [ ]:
"""Module with transforming of dataset to Coco format"""

import json
import os.path
from pathlib import Path
from typing import Any, Dict, List, Tuple, Union

import pandas as pd
from tqdm import tqdm


class DatasetToCocoTransformer:
    ANNOTATIONS_COLUMN = 'annotations'
    VIDEO_ID_COLUMN = 'video_id'
    VIDEO_FRAME_COLUMN = 'video_frame'

    def __init__(
        self,
        annotation_dataframe: pd.DataFrame,
        image_extension: str = '.jpg',
        image_size: Tuple[int, int] = (1280, 720),
        verbose: bool = True,
    ) -> None:
        self._annotation_dataframe = annotation_dataframe

        self._image_extension = image_extension
        self._image_size = image_size
        self._verbose = verbose

    def transform(self) -> Dict[str, Any]:
        self._preprocess_dataframe()
        width_image, height_image = self._image_size

        images_info = []
        annotations_info = []
        categories_info = [{'supercategory': 'cot', 'id': 0, 'name': 'cot'}]
        last_segmentation_id = -1

        for index, curr_row in tqdm(
            self._annotation_dataframe.iterrows(),
            postfix='Transforming dataset...',
            disable=not self._verbose,
            total=self._annotation_dataframe.shape[0],
        ):
            annotations = curr_row[self.ANNOTATIONS_COLUMN]
            video_id = curr_row[self.VIDEO_ID_COLUMN]
            video_frame = curr_row[self.VIDEO_FRAME_COLUMN]

            if len(annotations) > 0:
                image_info = self._get_image_info(
                    image_id=index,
                    video_id=video_id,
                    video_frame=video_frame,
                    width_image=width_image,
                    height_image=height_image,
                )
                (
                    image_annotations_info,
                    last_segmentation_id,
                ) = self._get_image_annotations_info(
                    image_id=index,
                    last_segmentation_id=last_segmentation_id,
                    coords=annotations,
                )

                images_info.append(image_info)
                annotations_info.extend(image_annotations_info)

        result = {
            'images': images_info,
            'annotations': annotations_info,
            'categories': categories_info,
        }

        return result

    @staticmethod
    def save_json(obj: Any, path: Path) -> None:
        with path.open(mode='w', encoding='UTF-8') as file:
            json.dump(obj, fp=file)

    def _get_image_info(
        self,
        image_id: int,
        video_id: int,
        video_frame: int,
        width_image: int,
        height_image: int,
    ) -> Dict[str, Any]:
        image_name = str(video_frame) + self._image_extension
        image_rel_path = os.path.join(f'video_{str(video_id)}', image_name)

        image_info = {
            'file_name': image_rel_path,
            'width': width_image,
            'height': height_image,
            'id': image_id,
        }

        return image_info

    def _get_image_annotations_info(
        self, image_id: int, last_segmentation_id: int, coords: List[Dict[str, int]]
    ) -> Tuple[List[Dict[str, Any]], int]:
        image_annotations = []

        for curr_coords in coords:
            transformed_coords = self._transform_bbox(bbox=curr_coords)

            last_segmentation_id += 1
            curr_image_annotation = {
                'bbox': transformed_coords,
                'category_id': 0,
                'image_id': image_id,
                'iscrowd': False,
                'area': transformed_coords[2] * transformed_coords[3],
                'id': last_segmentation_id,
            }
            image_annotations.append(curr_image_annotation)

        return image_annotations, last_segmentation_id

    def _preprocess_dataframe(self) -> None:
        if isinstance(self._annotation_dataframe[self.ANNOTATIONS_COLUMN].loc[0], str):
            self._annotation_dataframe[
                self.ANNOTATIONS_COLUMN
            ] = self._annotation_dataframe[self.ANNOTATIONS_COLUMN].apply(eval)

        self._annotation_dataframe[self.VIDEO_ID_COLUMN] = self._annotation_dataframe[
            self.VIDEO_ID_COLUMN
        ].apply(int)
        self._annotation_dataframe[
            self.VIDEO_FRAME_COLUMN
        ] = self._annotation_dataframe[self.VIDEO_FRAME_COLUMN].apply(int)

    @staticmethod
    def _transform_bbox(bbox: Dict[str, int]) -> List[float]:
        x_min = int(bbox['x'])
        y_min = int(bbox['y'])
        width = int(bbox['width'])
        height = int(bbox['height'])

        return [x_min, y_min, width, height]


In [ ]:
"""Module with CLI for converting dataset to YoloV5 format"""


from pathlib import Path
from typing import Tuple

import pandas as pd


def transform_dataset_to_coco(
    annotations_path: Path,
    res_path: Path,
    image_extension: str = '.jpg',
    image_size: Tuple[int, int] = (1280, 720),
    verbose: bool = True,
) -> None:
    dataframe = pd.read_csv(annotations_path)

    dataset_to_yolo_transformer = DatasetToCocoTransformer(
        annotation_dataframe=dataframe,
        image_extension=image_extension,
        image_size=image_size,
        verbose=verbose,
    )

    result = dataset_to_yolo_transformer.transform()
    dataset_to_yolo_transformer.save_json(result, res_path)


## Let's split data

Using code above you can split data in two ways:

1. By length. In current state last 30% of each video would be sent to validation.
2. By video. In current state `video_0` and `video_1` would be used for trainig, `video_2` for validation.

You can change `SPLIT_TYPE` argument below to `SplitType.video`, to use second way.

In [ ]:
DATAFRAME_PATH = Path('../input/tensorflow-great-barrier-reef/train.csv')
SPLIT_TYPE = SplitType.length

TRAIN_NAME = 'train_part.csv'
VAL_NAME = 'val_part.csv'

split_dataframe(
    original_dataframe_path=DATAFRAME_PATH,
    train_name=TRAIN_NAME,
    val_name=VAL_NAME,
    split_type=SPLIT_TYPE,
)

## Let's transform data

YOLOX uses COCO-like dataset, so lets transform our splits to COCO annotations

In [ ]:
TRAIN_RES_COCO_PATH = Path('train.json')

transform_dataset_to_coco(
    annotations_path=Path(TRAIN_NAME),
    res_path=Path(TRAIN_RES_COCO_PATH)
)

VAL_RES_COCO_PATH = Path('val.json')

transform_dataset_to_coco(
    annotations_path=Path(VAL_NAME),
    res_path=Path(VAL_RES_COCO_PATH)
)

Now you have two files: `./train.json` and `./val.json`. You can use them to train your YOLOX!

If you've found a bug, let me know!

## If this notebook was helpful, please upvote!